In [1]:
import sys

sys.path.append("/home/user/Documents/projects/osil")

import os
import importlib
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cv2
import copy

import projects.metaworld.dataset as metaworld_dataset

import utils.camera_utils_v2 as cu
import sensors.point_cloud_sensor as pc_sensor
from scipy.interpolate import RegularGridInterpolator

# for feature extraction and matching
# from lightglue import LightGlue, SuperPoint, DISK
import models.lightglue as lightglue_train_model
import torch

# torch.set_grad_enabled(False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'

In [33]:
# torch.set_grad_enabled(True)

In [2]:
importlib.reload(metaworld_dataset)

<module 'projects.metaworld.dataset' from '/home/user/Documents/projects/osil/projects/metaworld/dataset.py'>

In [3]:
EXCLUDED_ENV_IDS = [1,22,31,41,42]
NUM_ENVS = 50

env_ids = []
for i in range(NUM_ENVS):
    if i not in EXCLUDED_ENV_IDS:
        env_ids.append(i)
print(env_ids)

dataset = metaworld_dataset.MetaworldFeatureMatchingDataset(
    data_folder="/media/user/ssd2t/datasets2/metaworld_keyframes/all_envs",
    max_keypoints=50,
    env_ids=env_ids,
    eps=0.01,
    dev=True
)

[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 45, 46, 47, 48, 49]


In [6]:
s = np.random.randint(0, len(dataset))
img_0, img_1, depth_0, depth_1, seg_0, seg_1, match_data, kpts_0, kpts_1, dscpt_0, dscpt_1, matches_0, matches_1, assignment_mtr = dataset[s]
# match_utils.visualize_matches_with_imgs(img_0.numpy(), img_1.numpy(), match_data)

In [7]:
model = lightglue_train_model.LightGlue({})
model.to(device)
model.train()


LightGlue(
  (input_proj): Identity()
  (posenc): LearnableFourierPositionalEncoding(
    (Wr): Linear(in_features=2, out_features=32, bias=False)
  )
  (transformers): ModuleList(
    (0-8): 9 x TransformerLayer(
      (self_attn): SelfBlock(
        (Wqkv): Linear(in_features=256, out_features=768, bias=True)
        (inner_attn): Attention()
        (out_proj): Linear(in_features=256, out_features=256, bias=True)
        (ffn): Sequential(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (2): GELU(approximate='none')
          (3): Linear(in_features=512, out_features=256, bias=True)
        )
      )
      (cross_attn): CrossBlock(
        (to_qk): Linear(in_features=256, out_features=256, bias=True)
        (to_v): Linear(in_features=256, out_features=256, bias=True)
        (to_out): Linear(in_features=256, out_features=256, bias=True)
        (ffn): Sequential(
          (0): Linear

In [8]:
data = {
    "keypoints0": kpts_0[None].to(device),
    "descriptors0": dscpt_0[None].to(device),
    "keypoints1": kpts_1[None].to(device),
    "descriptors1": dscpt_1[None].to(device),
    "view0": {
        "image_size": torch.Tensor([360, 480]).long().to(device)
    },
    "view1":{
        "image_size": torch.Tensor([360, 480]).long().to(device)
    },
    "gt_matches0": matches_0[None].to(device),
    "gt_matches1": matches_1[None].to(device),
    "gt_assignment": assignment_mtr[None].to(device),
}
pred = model(data)
losses, metrics = model.loss(pred, data)
loss = torch.mean(losses["total"])

In [12]:
metrics.keys()

dict_keys([])

In [10]:
losses.keys()

dict_keys(['total', 'last', 'assignment_nll', 'nll_pos', 'nll_neg', 'num_matchable', 'num_unmatchable', 'confidence', 'row_norm'])

In [9]:
loss.backward()

In [12]:
loss["total"].

({'total': tensor([6.4842], device='cuda:0'),
  'last': tensor([6.1328], device='cuda:0'),
  'assignment_nll': tensor([6.1328], device='cuda:0'),
  'nll_pos': tensor([11.0988], device='cuda:0'),
  'nll_neg': tensor([1.1668], device='cuda:0'),
  'num_matchable': tensor([7656.], device='cuda:0'),
  'num_unmatchable': tensor([20.5000], device='cuda:0'),
  'confidence': tensor([0.6971], device='cuda:0'),
  'row_norm': tensor([0.3949], device='cuda:0')},
 {})